In [0]:
%fs
ls "/mnt/creditcard1"

path,name,size,modificationTime
dbfs:/mnt/creditcard1/credit_raw_data/,credit_raw_data/,0,1750330314000
dbfs:/mnt/creditcard1/credit_transformed_data/,credit_transformed_data/,0,1750330337000
dbfs:/mnt/creditcard1/tf-data/,tf-data/,0,1750406426000


In [0]:
%fs
ls "dbfs:/mnt/creditcard1/credit_raw_data/"

path,name,size,modificationTime
dbfs:/mnt/creditcard1/credit_raw_data/credit_raw_data.csv,credit_raw_data.csv,4338154,1750333127000


In [0]:
credit = spark.read.csv("dbfs:/mnt/creditcard1/credit_raw_data/credit_raw_data.csv", header=True)

In [0]:
credit.show()

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+--------------------------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|default payment next month|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+--------------------------+
|  1|    20000|  2|        2|       1| 24|    2|    2|    0|    0|    0|    0|     3913|     3102|      689|        0|        0|        0|       0|     689|       0|       0|       0|       0|                         1|
|  2|   120000|  2|        2|       2| 26|    0|    2|    0|    0|    0|    2|     2682|     1725|     2682|     3272|  

In [0]:
credit.printSchema()



root
 |-- ID: string (nullable = true)
 |-- LIMIT_BAL: string (nullable = true)
 |-- SEX: string (nullable = true)
 |-- EDUCATION: string (nullable = true)
 |-- MARRIAGE: string (nullable = true)
 |-- AGE: string (nullable = true)
 |-- PAY_0: string (nullable = true)
 |-- PAY_2: string (nullable = true)
 |-- PAY_3: string (nullable = true)
 |-- PAY_4: string (nullable = true)
 |-- PAY_5: string (nullable = true)
 |-- PAY_6: string (nullable = true)
 |-- BILL_AMT1: string (nullable = true)
 |-- BILL_AMT2: string (nullable = true)
 |-- BILL_AMT3: string (nullable = true)
 |-- BILL_AMT4: string (nullable = true)
 |-- BILL_AMT5: string (nullable = true)
 |-- BILL_AMT6: string (nullable = true)
 |-- PAY_AMT1: string (nullable = true)
 |-- PAY_AMT2: string (nullable = true)
 |-- PAY_AMT3: string (nullable = true)
 |-- PAY_AMT4: string (nullable = true)
 |-- PAY_AMT5: string (nullable = true)
 |-- PAY_AMT6: string (nullable = true)
 |-- default payment next month: string (nullable = true)



CASTING ALL NUMERIC DATA TO DOUBLE DATA TYPE FROM STRING


In [0]:
from pyspark.sql.functions import col

numeric_columns = [
    'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE',
    'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6',
    'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
    'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
    'default payment next month'
]

credit = credit.select([col(c).cast("double") if c in numeric_columns else col(c) for c in credit.columns])


In [0]:
# Optional: filter out invalid education/marriage values
credit = credit.filter((col("EDUCATION").isin(1.0, 2.0, 3.0, 4.0)) & (col("MARRIAGE").isin(1.0, 2.0, 3.0)))

In [0]:
#CHECKING HOW MANY NULL VALUES EXIST PER COLUMN
from pyspark.sql.functions import col, sum as _sum, when

null_counts = credit.select([
    _sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in credit.columns
])

null_counts.show()


+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+--------------------------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|default payment next month|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+--------------------------+
|  0|        0|  0|        0|       0|  0|    0|    0|    0|    0|    0|    0|        0|        0|        0|        0|        0|        0|       0|       0|       0|       0|       0|       0|                         0|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+--

total number of null values = 0

Feature Engineering

In [0]:
from pyspark.sql.functions import sum as _sum, expr

credit = credit.withColumn("TOTAL_BILL", 
                           col("BILL_AMT1") + col("BILL_AMT2") + col("BILL_AMT3") +
                           col("BILL_AMT4") + col("BILL_AMT5") + col("BILL_AMT6"))

credit = credit.withColumn("TOTAL_PAY", 
                           col("PAY_AMT1") + col("PAY_AMT2") + col("PAY_AMT3") +
                           col("PAY_AMT4") + col("PAY_AMT5") + col("PAY_AMT6"))

credit = credit.withColumn("PAY_RATIO", expr("case when TOTAL_BILL > 0 then TOTAL_PAY / TOTAL_BILL else 0 end"))


SKEWNESS CHECK

In [0]:
from pyspark.sql.functions import skewness

columns_to_check = [
    'LIMIT_BAL', 'AGE',
    'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 
    'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6',
    'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 
    'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'
]

# Optional: include engineered features if already created
columns_to_check += ['TOTAL_BILL', 'TOTAL_PAY', 'PAY_RATIO']

credit.select([skewness(col(c)).alias(c + "_skew") for c in columns_to_check]).show()


+----------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+
|  LIMIT_BAL_skew|          AGE_skew|    BILL_AMT1_skew|    BILL_AMT2_skew|    BILL_AMT3_skew|    BILL_AMT4_skew|   BILL_AMT5_skew|   BILL_AMT6_skew|     PAY_AMT1_skew|     PAY_AMT2_skew|     PAY_AMT3_skew|     PAY_AMT4_skew|     PAY_AMT5_skew|     PAY_AMT6_skew|   TOTAL_BILL_skew|   TOTAL_PAY_skew|    PAY_RATIO_skew|
+----------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+
|0.99279955637722|0.7372720418203172|2.6

SKEWNESS ANALYSIS

LIMIT_BAL	~0.99	Moderately right-skewed
AGE	~0.73	Slight right-skew
BILL_AMT1-6	~2.6 – 3.1	Highly right-skewed
PAY_AMT1-6	~10 – 30 	Extremely right-skewed
TOTAL_BILL	~2.74	Highly right-skewed
TOTAL_PAY	~14.77	Extremely right-skewed
PAY_RATIO	114.65	Extremely right-skewed

SINCE WE WOULD REQUIRE ORIGINAL DATA FOR POWER BI IMPLEMENTATION , NOT ADDING NON SKEW VALUES 

In [0]:
credit = credit.dropna()

In [0]:
credit = credit.filter(credit['LIMIT_BAL'] >= 10000)



In [0]:
credit.show()

+---+---------+---+---------+--------+----+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+--------------------------+----------+---------+--------------------+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE| AGE|PAY_0|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|default payment next month|TOTAL_BILL|TOTAL_PAY|           PAY_RATIO|
+---+---------+---+---------+--------+----+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+--------------------------+----------+---------+--------------------+
|  1|  20000.0|2.0|      2.0|     1.0|24.0|  2.0|  2.0|  0.0|  0.0|  0.0|  0.0|   3913.0|   3102.0|    689.0|      0.0|      0.0|      0.0|     0.0|   689.0|     0.0|     0.0|     0.0|     0.0|                  

In [0]:
credit.count()

29601

FOR BETTER READABILITY

In [0]:
credit = credit.withColumnRenamed("LIMIT_BAL", "Credit_Limit") \
               .withColumnRenamed("SEX", "Gender") \
               .withColumnRenamed("EDUCATION", "Education_Level") \
               .withColumnRenamed("MARRIAGE", "Marital_Status") \
               .withColumnRenamed("AGE", "Age") \
               .withColumnRenamed("default payment next month", "Default_Status")


In [0]:
credit.write.mode("overwrite").option("header", True).csv("/mnt/creditcard1/processed/transformed_data.csv")
